* 이 커널은 [Titanic Top 4% with ensemble modeling](https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling) 커널을 필사하며 정리한 것입니다.

# Titanic Top 4% with ensemble modeling

* **1 Introduction**

* **2 Load and check data**<br>
 * 2.1 load data
 * 2.2 Outlier detection
 * 2.3 joining train and test set
 * 2.4 check for null and missing values

* **3 Feature analysis**
 * 3.1 Numerical values
 * 3.2 Categorical values

* **4 Filling missing Values**
 * 4.1 Age

* **5 Feature engineering**
 * 5.1 Name/Title
 * 5.2 Family Size
 * 5.3 Cabin
 * 5.4 Ticket

* **6 Modeling**
 * 6.1 Simple modeling
   * 6.1.1 Cross validate models
   * 6.1.2 Hyperparamater tunning for best models
   * 6.1.3 Plot learning curves
   * 6.1.4 Feature importance of the tree based classifiers
 * 6.2 Ensemble modeling
   * 6.2.1 Combining models
 * 6.3 Prediction
   * 6.3.1 Predict and Submit results

<br><br>

## 1. Introduction
이 커널은 크게 세 가지 파트로 나뉩니다.

* **Feature analysis**
* **Feature engineering**
* **Modeling**

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')

## 2. Load and check data
### 2.1 Load data

In [0]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
IDtest = test["PassengerId"]

### 2.2 Outlier detection

In [0]:
# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])